In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input
from tensorflow.keras import Model

import tempfile, sys, os
sys.path.insert(0, os.path.abspath('..'))

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [4]:
from sam import sam_train_step

class MyModel(Model):
    def train_step(self, data):
        return sam_train_step(self, data)

In [5]:
inputs = Input(shape=(28,28,1))
x = Conv2D(32, 3, activation='relu')(inputs)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(10)(x)
model = MyModel(inputs, outputs)

In [6]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [7]:
model.fit(x_train, y_train, epochs=3)

Epoch 1/3
1875/1875 [==============================] - 84s 44ms/step - loss: 0.3142 - accuracy: 0.9056
Epoch 2/3
1875/1875 [==============================] - 81s 43ms/step - loss: 0.0559 - accuracy: 0.9826
Epoch 3/3
1875/1875 [==============================] - 82s 43ms/step - loss: 0.0351 - accuracy: 0.9896
